In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import  tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from sklearn.preprocessing import StandardScaler
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
df = data

In [ ]:
plt.title('Count of Stroke')
sns.countplot(x ='stroke',  data = df)

In [ ]:
df['bmi'] = df['bmi'].fillna(method = 'ffill')

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
col = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for col in col:
    df[col] = le.fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop(['id'],axis=1)
X = df.iloc[:,0:10]
y = df['stroke']

In [ ]:
X.head()

In [ ]:
'''
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=0) 
X, y = ros.fit_resample(X, y)
Counter(y)
'''

In [ ]:
#标准化及编码
biaozhunhua = StandardScaler()
X = biaozhunhua.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=11)

y_train_OneHot = tf.keras.utils.to_categorical(y_train)
y_test_OneHot = tf.keras.utils.to_categorical(y_test)


In [ ]:
plt.figure(figsize=(18,6))
fig = sns.boxplot(x="variable", y="value", data=pd.melt(frame=df))
plt.grid()
plt.show()

In [ ]:
#网络建造
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
#加入Dropout功能避免过度拟合
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(2,activation='sigmoid'))

model.build(input_shape=(4,10))

#查看模型摘要+++++++++++++++++++++++++++++++++++++++++++
print(model.summary())


In [ ]:
# 选择优化器

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',metrics=['accuracy']) 

t1=time.time()
train_history=model.fit(x=X_train,y=y_train_OneHot,validation_split=0.2, 
                        epochs=20, batch_size=64,verbose=2)
t2=time.time()
CNNfit = float(t2-t1)
print("Time taken: {} seconds".format(CNNfit))


In [ ]:
#可视化准确率与误差
import matplotlib.pyplot as plt
def show_train_history(train_acc,test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# 训练与验证曲线
show_train_history('accuracy','val_accuracy')  #准确率执行结果
show_train_history('loss','val_loss')
